# Get KFC locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import pandas as pd
import altair as alt
import geopandas as gpd
from tqdm.notebook import tqdm, trange
from vega_datasets import data
from time import sleep
from random import randint

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
place = "KFC"

In [5]:
cookies = {
    "bm_sz": "C389C4FA5FAF5E02ED3A48D609C53920~YAAQjiABFzbzT1yNAQAAL0eQkBYt/d/pQ3YGI8sqN9hSYqtFCwtBCGFLGVa7tXxuf6PvL804qqSxjjuxaRoS1FDMSwol4Eg9gVsncl/zIMQuMxmNIxDc3P47rsgZ7qoxa3hmfI9pQBwRZEMYhDu1qjI9fj4AVcnGsfMOqgpH1VKLZvHk75SMwQC8w/HlMzb49FoS5EmJ4doyUHCCOTREMUwidMIeRh+WMX6Ke7BAUVhifizw+SDkp9yy4FalHEHinjU2hkIESstxusDSZRPxNnE3EKVjIN8xFNdJcqMlQZ5cv6OZNRbxtjkGIMJckqeJ4XxJQ00u3Z1Yx2Gl/w==~4403249~3621700",
    "PIM-SESSION-ID": "ynueFPBPg5NXZ4Lb",
    "optimizelyEndUserId": "oeu1707527392056r0.385011537816075",
    "AMP_MKTG_0b6137762d": "JTdCJTIycmVmZXJyZXIlMjIlM0ElMjJodHRwcyUzQSUyRiUyRnd3dy5nb29nbGUuY29tJTJGJTIyJTJDJTIycmVmZXJyaW5nX2RvbWFpbiUyMiUzQSUyMnd3dy5nb29nbGUuY29tJTIyJTdE",
    "OTGPPConsent": "DBABLA~BVQqAAAACZA.QA",
    "AMP_0b6137762d": "JTdCJTIyZGV2aWNlSWQlMjIlM0ElMjIyNTkyZDM5Yi1lNWU0LTRkY2ItOTgyZS00MjUzNzk1MTIwMWYlMjIlMkMlMjJzZXNzaW9uSWQlMjIlM0ExNzA3NTI3Mzk1NTE0JTJDJTIyb3B0T3V0JTIyJTNBZmFsc2UlMkMlMjJsYXN0RXZlbnRUaW1lJTIyJTNBMTcwNzUyNzM5NjA4NCU3RA==",
    "ak_bmsc": "15A668ECE6CF35A89763FCC9C326D061~000000000000000000000000000000~YAAQjiABF3v1T1yNAQAAYFuQkBbov97JowRmt4wlbdcOYY+vCr83f4BuO4pCIQ0FPzVyUk6ZnyB1AkbwhI/Q+ZAhI8j3GHggF2C2hq9myk8IkZiEAuvzMvgZjegyPfA7VDu08uoC07eOqAjyRvzs3om+9tXpZ9J+MBvDy8YLzazkgKwSD6OLm4p/XBqUapnz+yC+2WEU6R8L9ccUQUMAjfpnWr2gJaPzqMCioSofaqdQur5onygYOFFlGMSNI65TRZatfIzvXrIr6Gp1zzQoepyiabctyaWGZVg4pmhdpy0GeZ8HHT08a3xRWNEBnaDmA11lJe66KTZggkJtL94iZSZSrfxj0e2nTqUBKHEMFt0xdDW3/d2HqkpZPP0wL6rxselsyhP5oTLomDKAVVlRLzGF5zclvHxRPJEv4K9hWkkCInAJ82xIF3FI8wDDrV4R7jBpBcutfm8z2pn9hAuiEeEs3phSZrCMbdQItdXrjCTVOs55qRYEo5Lcclj+z78=",
    "_gcl_au": "1.1.1053695645.1707527396",
    "bm_sv": "0623CA4CD14580A6949987750BE1C6DA~YAAQjiABF8L1T1yNAQAAxV2QkBZhFYwx4NJt79WrU7AssU3DzjUzT9OCivPkkhjFy93+noBkay3FpfgrcmxhKZsBLk2Nf0RdKWWje2PUOo0RFDEd4slHc2RFaEvCt9lItE5yJdroaUU3DjHKh0HAtJuwLBn8u1d2OemF6HGIqcdbK4vYKMLKTW9NGJHnnv3K1NlHvG5RbWdKykoV8+3nefCzrZaRiF306qEyFTVQaGb6mxkH1C3rqzrxO8gL~1",
    "ab.storage.sessionId.f5bc3d2d-256d-4e1b-b061-f6b182d94c39": "%7B%22g%22%3A%229634176f-38ba-e4eb-3e42-432148a0fa9c%22%2C%22e%22%3A1707529197235%2C%22c%22%3A1707527397235%2C%22l%22%3A1707527397235%7D",
    "ab.storage.userId.f5bc3d2d-256d-4e1b-b061-f6b182d94c39": "%7B%22g%22%3A%22-8179047820597560240%22%2C%22c%22%3A1707527397234%2C%22l%22%3A1707527397235%7D",
    "ab.storage.deviceId.f5bc3d2d-256d-4e1b-b061-f6b182d94c39": "%7B%22g%22%3A%229cfad3d4-8c71-c6b0-d803-b191a7c449e7%22%2C%22c%22%3A1707527397236%2C%22l%22%3A1707527397236%7D",
    "_tt_enable_cookie": "1",
    "_ttp": "wJkMLbaaZ1zBw4xnnhglZkpgLyG",
    "_derived_epik": "dj0yJnU9VXlRc25GWVl0dmozLUNLTTV2b1RiX3BlZXJLejEyQlcmbj1nSVNkWExhbEVvRHdXVEpSY2o3OGt3Jm09MSZ0PUFBQUFBR1hHek9VJnJtPTEmcnQ9QUFBQUFHWEd6T1Umc3A9Mg",
    "_pin_unauth": "dWlkPU5UQXdaRGd3WlRNdE5qRTVNeTAwWWpkaUxUazNPVGt0T0dKak0yTmxOR014WkdZeA",
    "OptanonAlertBoxClosed": "2024-02-10T01:10:07.482Z",
    "OptanonConsent": "isGpcEnabled=0&datestamp=Fri+Feb+09+2024+17%3A10%3A07+GMT-0800+(Pacific+Standard+Time)&version=202401.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=9d8bb8d6-5758-437d-9e72-50cc5d5e7d84&interactionCount=1&landingPath=NotLandingPage&GPPCookiesCount=1&groups=C0002%3A1%2CC0004%3A1%2COSSTA_BG%3A1",
    "_abck": "8B0B15C59EA2EA32C1BDD6449F5CD8AA~-1~YAAQjiABF0MAUFyNAQAAHKqQkAuGMBbNCGggYPCkhe+BVYhuWtF8kCoaCwKFlvpbkWr7at0nMaQHsmqw4N0R7TcMkxlt4BLZvWJ9DEjW/fsMKzxsMXjbjTKyUgXJ3SXz1QnyMH5wLkPekhpA9sg6P/cBK3xBv1VnhUKWDNlD2YM2gwxl1tlFE2/6syLMUbykSpdv5/F8IZ5MUqJJHVAgTPkrhTmq65xviTW3l7Hjn4PRxjNrQg7DSRXiUL+7Pt8+tXvoCyBz9UjvVFr8NuxRLa4vPQhJu5bwaCsJq3eg+QoFocKJR04NH9s268uKP8l0MX7padLgoiRee4uWj/3YBJsdKgXjbXp/9eHsCBpquQy2vYDDgXDUESLfif9CRWtZSskKTECIB8BfcftahKsw1ysK2EA4HfURNo51dkVIQj2FYQ==~-1~-1~1707530921",
    "_ga_F1Z56ZPP4C": "GS1.1.1707527396.1.1.1707527416.40.0.0",
    "__cf_bm": "rbEEnuyWGwc6PkJpkxOMLrCO.9JYXIwGSun2hQIPUuM-1707527416-1-AS+n5i56qTrxxRhC5yRVcjAaVognMoIMgkcGaNZ/wFxGPlZDcIK7aWD/FxofKd7ofe0J4MoqKVGj72IG3pI5dQs=",
    "_yfpc": "1397504499727",
    "_ga": "GA1.2.1973205296.1707527397",
    "_gid": "GA1.2.376998104.1707527417",
    "_gat_kfc": "1",
    "_gat_yext": "1",
    "_ga_D9CXGTW3EF": "GS1.2.1707527417.1.1.1707527563.0.0.0",
    "_ga_PTKQ8DC73S": "GS1.2.1707527417.1.1.1707527563.60.0.0",
}

headers = {
    "authority": "locations.kfc.com",
    "accept": "application/json",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    # 'cookie': 'bm_sz=C389C4FA5FAF5E02ED3A48D609C53920~YAAQjiABFzbzT1yNAQAAL0eQkBYt/d/pQ3YGI8sqN9hSYqtFCwtBCGFLGVa7tXxuf6PvL804qqSxjjuxaRoS1FDMSwol4Eg9gVsncl/zIMQuMxmNIxDc3P47rsgZ7qoxa3hmfI9pQBwRZEMYhDu1qjI9fj4AVcnGsfMOqgpH1VKLZvHk75SMwQC8w/HlMzb49FoS5EmJ4doyUHCCOTREMUwidMIeRh+WMX6Ke7BAUVhifizw+SDkp9yy4FalHEHinjU2hkIESstxusDSZRPxNnE3EKVjIN8xFNdJcqMlQZ5cv6OZNRbxtjkGIMJckqeJ4XxJQ00u3Z1Yx2Gl/w==~4403249~3621700; PIM-SESSION-ID=ynueFPBPg5NXZ4Lb; optimizelyEndUserId=oeu1707527392056r0.385011537816075; AMP_MKTG_0b6137762d=JTdCJTIycmVmZXJyZXIlMjIlM0ElMjJodHRwcyUzQSUyRiUyRnd3dy5nb29nbGUuY29tJTJGJTIyJTJDJTIycmVmZXJyaW5nX2RvbWFpbiUyMiUzQSUyMnd3dy5nb29nbGUuY29tJTIyJTdE; OTGPPConsent=DBABLA~BVQqAAAACZA.QA; AMP_0b6137762d=JTdCJTIyZGV2aWNlSWQlMjIlM0ElMjIyNTkyZDM5Yi1lNWU0LTRkY2ItOTgyZS00MjUzNzk1MTIwMWYlMjIlMkMlMjJzZXNzaW9uSWQlMjIlM0ExNzA3NTI3Mzk1NTE0JTJDJTIyb3B0T3V0JTIyJTNBZmFsc2UlMkMlMjJsYXN0RXZlbnRUaW1lJTIyJTNBMTcwNzUyNzM5NjA4NCU3RA==; ak_bmsc=15A668ECE6CF35A89763FCC9C326D061~000000000000000000000000000000~YAAQjiABF3v1T1yNAQAAYFuQkBbov97JowRmt4wlbdcOYY+vCr83f4BuO4pCIQ0FPzVyUk6ZnyB1AkbwhI/Q+ZAhI8j3GHggF2C2hq9myk8IkZiEAuvzMvgZjegyPfA7VDu08uoC07eOqAjyRvzs3om+9tXpZ9J+MBvDy8YLzazkgKwSD6OLm4p/XBqUapnz+yC+2WEU6R8L9ccUQUMAjfpnWr2gJaPzqMCioSofaqdQur5onygYOFFlGMSNI65TRZatfIzvXrIr6Gp1zzQoepyiabctyaWGZVg4pmhdpy0GeZ8HHT08a3xRWNEBnaDmA11lJe66KTZggkJtL94iZSZSrfxj0e2nTqUBKHEMFt0xdDW3/d2HqkpZPP0wL6rxselsyhP5oTLomDKAVVlRLzGF5zclvHxRPJEv4K9hWkkCInAJ82xIF3FI8wDDrV4R7jBpBcutfm8z2pn9hAuiEeEs3phSZrCMbdQItdXrjCTVOs55qRYEo5Lcclj+z78=; _gcl_au=1.1.1053695645.1707527396; bm_sv=0623CA4CD14580A6949987750BE1C6DA~YAAQjiABF8L1T1yNAQAAxV2QkBZhFYwx4NJt79WrU7AssU3DzjUzT9OCivPkkhjFy93+noBkay3FpfgrcmxhKZsBLk2Nf0RdKWWje2PUOo0RFDEd4slHc2RFaEvCt9lItE5yJdroaUU3DjHKh0HAtJuwLBn8u1d2OemF6HGIqcdbK4vYKMLKTW9NGJHnnv3K1NlHvG5RbWdKykoV8+3nefCzrZaRiF306qEyFTVQaGb6mxkH1C3rqzrxO8gL~1; ab.storage.sessionId.f5bc3d2d-256d-4e1b-b061-f6b182d94c39=%7B%22g%22%3A%229634176f-38ba-e4eb-3e42-432148a0fa9c%22%2C%22e%22%3A1707529197235%2C%22c%22%3A1707527397235%2C%22l%22%3A1707527397235%7D; ab.storage.userId.f5bc3d2d-256d-4e1b-b061-f6b182d94c39=%7B%22g%22%3A%22-8179047820597560240%22%2C%22c%22%3A1707527397234%2C%22l%22%3A1707527397235%7D; ab.storage.deviceId.f5bc3d2d-256d-4e1b-b061-f6b182d94c39=%7B%22g%22%3A%229cfad3d4-8c71-c6b0-d803-b191a7c449e7%22%2C%22c%22%3A1707527397236%2C%22l%22%3A1707527397236%7D; _tt_enable_cookie=1; _ttp=wJkMLbaaZ1zBw4xnnhglZkpgLyG; _derived_epik=dj0yJnU9VXlRc25GWVl0dmozLUNLTTV2b1RiX3BlZXJLejEyQlcmbj1nSVNkWExhbEVvRHdXVEpSY2o3OGt3Jm09MSZ0PUFBQUFBR1hHek9VJnJtPTEmcnQ9QUFBQUFHWEd6T1Umc3A9Mg; _pin_unauth=dWlkPU5UQXdaRGd3WlRNdE5qRTVNeTAwWWpkaUxUazNPVGt0T0dKak0yTmxOR014WkdZeA; OptanonAlertBoxClosed=2024-02-10T01:10:07.482Z; OptanonConsent=isGpcEnabled=0&datestamp=Fri+Feb+09+2024+17%3A10%3A07+GMT-0800+(Pacific+Standard+Time)&version=202401.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=9d8bb8d6-5758-437d-9e72-50cc5d5e7d84&interactionCount=1&landingPath=NotLandingPage&GPPCookiesCount=1&groups=C0002%3A1%2CC0004%3A1%2COSSTA_BG%3A1; _abck=8B0B15C59EA2EA32C1BDD6449F5CD8AA~-1~YAAQjiABF0MAUFyNAQAAHKqQkAuGMBbNCGggYPCkhe+BVYhuWtF8kCoaCwKFlvpbkWr7at0nMaQHsmqw4N0R7TcMkxlt4BLZvWJ9DEjW/fsMKzxsMXjbjTKyUgXJ3SXz1QnyMH5wLkPekhpA9sg6P/cBK3xBv1VnhUKWDNlD2YM2gwxl1tlFE2/6syLMUbykSpdv5/F8IZ5MUqJJHVAgTPkrhTmq65xviTW3l7Hjn4PRxjNrQg7DSRXiUL+7Pt8+tXvoCyBz9UjvVFr8NuxRLa4vPQhJu5bwaCsJq3eg+QoFocKJR04NH9s268uKP8l0MX7padLgoiRee4uWj/3YBJsdKgXjbXp/9eHsCBpquQy2vYDDgXDUESLfif9CRWtZSskKTECIB8BfcftahKsw1ysK2EA4HfURNo51dkVIQj2FYQ==~-1~-1~1707530921; _ga_F1Z56ZPP4C=GS1.1.1707527396.1.1.1707527416.40.0.0; __cf_bm=rbEEnuyWGwc6PkJpkxOMLrCO.9JYXIwGSun2hQIPUuM-1707527416-1-AS+n5i56qTrxxRhC5yRVcjAaVognMoIMgkcGaNZ/wFxGPlZDcIK7aWD/FxofKd7ofe0J4MoqKVGj72IG3pI5dQs=; _yfpc=1397504499727; _ga=GA1.2.1973205296.1707527397; _gid=GA1.2.376998104.1707527417; _gat_kfc=1; _gat_yext=1; _ga_D9CXGTW3EF=GS1.2.1707527417.1.1.1707527563.0.0.0; _ga_PTKQ8DC73S=GS1.2.1707527417.1.1.1707527563.60.0.0',
    "referer": "https://locations.kfc.com/search?q=90066&country=&region=&qp=",
    "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

params = {
    "q": "",
    "country": "",
    "region": "",
    "qp": "",
    "offset": 10,
}

In [6]:
count = requests.get(
    "https://locations.kfc.com/search", params=params, headers=headers
).json()["response"]["count"]

In [7]:
count

4276

In [9]:
locations_list = []

for r in tqdm(range(0, count, 10)[1:]):
    params = {
        "q": "",
        "country": "",
        "region": "",
        "qp": "",
        "offset": r,
    }

    for req in requests.get(
        "https://locations.kfc.com/search", params=params, headers=headers
    ).json()["response"]["entities"]:
        address_dict = {
            "city": req["profile"]["address"]["city"],
            "street": req["profile"]["address"]["line1"],
            "state": req["profile"]["address"]["region"],
            "zip": req["profile"]["address"]["postalCode"],
            "country": req["profile"]["address"]["countryCode"],
            "latitude": req["profile"]["yextDisplayCoordinate"]["lat"],
            "longitude": req["profile"]["yextDisplayCoordinate"]["long"],
        }
        locations_list.append(address_dict)
        # sleep(randint(1,4))

  0%|          | 0/427 [00:00<?, ?it/s]

In [10]:
len(locations_list)

In [13]:
df = pd.DataFrame(locations_list)

---

## Geography

#### Make it a geodataframe

In [14]:
df_geo = df.copy()

In [15]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [16]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [17]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place} locations")
    .project("albersUsa")
)

#### Location points map

In [20]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=3, color="#E4002B")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [21]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("#E4002B"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [32]:
df.to_json(
    f"kfc/data/processed/{place.lower()}_locations.json", indent=4, orient="records"
)

#### CSV

In [31]:
df.to_csv(f"kfc/data/processed/{place.lower()}_locations.csv", index=False)

#### GeoJSON

In [33]:
locations_gdf.to_file(
    f"kfc/data/processed/{place.lower()}_locations.geojson", driver="GeoJSON"
)